In [ ]:
import pandas as pd
import sys
import numpy as np
from itertools import combinations
pd.set_option('display.max_rows', 100)

In [ ]:
sys.path.append('..\\packages')

In [ ]:
from suppression_check import DataAnonymizer

# CSV file with PreRedacted Data

In [ ]:
file_path = r'..\data\ParentChildTwoSensitive.csv'

# Load the CSV file into a DataFrame

In [ ]:
df_data = pd.read_csv(file_path)

# Instantiate the anonymizer

In [ ]:
parent_org = 'ParentEntity'

In [ ]:
child_org = 'ChildEntity'

In [ ]:
sensitive_list = ['Subgroup1', 'Subgroup2']

In [ ]:
frequency_value = 'GraduationCount'

In [ ]:
anonymizer = DataAnonymizer(df_data, parent_organization=parent_org, child_organization = child_org, sensitive_columns=sensitive_list, frequency=frequency_value)

In [ ]:
df_log = anonymizer.create_log()

In [ ]:
df_log.to_excel('1CreateLog.xlsx', index=False)

In [ ]:
df_less_than_threshold = anonymizer.less_than_threshold_not_zero()

In [ ]:
df_less_than_threshold.to_excel('2LessThanThreshold.xlsx', index=False)

In [ ]:
df_one_count = anonymizer.one_count_redacted()

In [ ]:
df_one_count.to_excel('3OneCountRedact.xlsx', index=False)

In [ ]:
df_log_na = df_one_count.copy()
temp_value = 'NaFill'
for sensitive_combination in self.sensitive_combinations:
    list_combination = list(sensitive_combination)
    if list_combination != self.sensitive_columns:
        string_combination = ''.join(list_combination)
        df_redact_less = df_log_na[df_log_na['RedactBinary'] == 1]
        df_redact_less['Redacted'] = 1
        df_count = df_redact_less.groupby(['Grouping'] + self.organization_columns + list_combination)['Redacted'].count().reset_index()
        df_one_redacted = df_count[df_count['Redacted'] == 1]
        if not df_one_redacted.empty:
            df_not_redacted = df_log_na[df_log_na['RedactBinary'] != 1]
            df_minimum = df_not_redacted.groupby(['Grouping'] + self.organization_columns + list_combination, dropna=False)['Counts'].min().reset_index()
            df_minimum.rename(columns={'Counts':'LastMiniumValue'}, inplace=True)
            df_minimum_redacted = df_one_redacted.merge(df_minimum, on = ['Grouping'] + self.organization_columns + list_combination)
            df_minimum_one = df_log_na.merge(df_minimum_redacted, on = ['Grouping'] + self.organization_columns + list_combination, how='left')
            mask = (df_minimum_one['Counts'] == df_minimum_one['LastMiniumValue'])
            df_log_na.loc[mask, 'RedactBinary'] = 1
            df_log_na.loc[mask, 'Redact'] = 'Secondary Suppression'
            df_log_na.loc[mask, 'RedactBreakdown'] += ', Redacting zeroes or other remaining values missed in one count function'

In [ ]:
df_one_redact_zero = anonymizer.one_redact_zero()

In [ ]:
df_one_redact_zero.to_excel('4OneRedactZero.xlsx', index=False)

In [ ]:
df_cross_suppression = anonymizer.cross_suppression()

In [ ]:
df_cross_suppression.to_excel('5CrossSuppression.xlsx', index=False)

In [ ]:
df_apply_log = anonymizer.apply_log()

In [ ]:
df_apply_log.to_excel('6ApplyLog.xlsx', index=False)